In [34]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import util.data_util as du
from math import cos, sin, pi, floor
import importlib
importlib.reload(du)

%matplotlib qt

HOBO = du.HOBOSensor("", "HOBO Farshore", "FieldDeployment_041621_Farshore.csv")

In [35]:
HOBO.import_data() # Import the HOBO data to memory
start_time = np.datetime64("2021-04-16T09:00:00")
end_time = start_time + np.timedelta64(20, 's')
start_index = np.where(HOBO.timestamp == start_time)[0][0]
end_index = np.where(HOBO.timestamp==end_time)[0][0]
window=(start_time, end_time)
windowed_time = HOBO.timestamp[start_index:end_index]

HOBO.plot_all_data(xlim=window)

## Section 2: 3D representation
Now, we are going to represent the Time, Amplitude, and Frequency domains in a simple 3-dimensional plot to illustrate how the sinusoidal waves stack together.

In [39]:
figure_fat = plt.figure(figsize=(12,12)) # Create a figure for the 3D representation of Frequency, Amplitude, and Time
ax_fat = figure_fat.add_subplot(projection='3d') # Create a 3D axis

# Create a plot of the surface elevation over time on the X-Z plane
t = np.divide(np.subtract(windowed_time, start_time), 1e9).astype(int) # Convert from datetime64 to integer from start_time (as 0) to end_time
ax_fat.plot(t, HOBO.surface[start_index:end_index], zdir='y')

# Create a plot of the surface elevation as a function of frequency on the Y-Z plane
freq = np.fft.fftfreq(len(t), t[1] - t[0])
# freq = np.linspace(0.05, 1/2, 20)
# print(freq)
surface_fft = np.fft.fft(HOBO.surface[start_index:end_index])

for n in range(len(surface_fft)):
    if n % 1 != 0 or freq[n] <= 0:
        continue
    recon_sig = np.real(surface_fft[n]) * np.cos(2*np.pi*freq[n]*t) - np.imag(surface_fft[n]) * np.sin(2*np.pi*freq[n]*t)
    ax_fat.plot(t, recon_sig, freq[n], zdir='y')

# Project the frequency domain
ax_fat.plot(freq, np.abs(surface_fft), zdir='x')

# Create a plane about the SWL (z=0)
xx, yy = np.meshgrid(range(20), np.arange(0, max(freq), 0.1))
# print((xx, yy))
ax_fat.plot_surface(xx, yy, np.zeros_like(xx), alpha=0.2)

# Prettify plot
ax_fat.set_ylim(0.05, max(freq))
ax_fat.view_init(15, -25, 0)
ax_fat.set_xlabel("Time [s]")
ax_fat.set_ylabel("Frequency [Hz]")
ax_fat.set_zlabel("Surface [m]")

plt.show()

### Section 2.1 - Sanity Check
We can sanity check our FFT by plotting the original time series against the reconstructed time series from the FFT

In [40]:
# Reconstruct each individual sine wave from the FFT
reconstructed_signal = np.zeros(len(t))
for n in range(floor(len(surface_fft)/2)-1):
    reconstructed_signal += (np.real(surface_fft[n]) * np.cos(2*np.pi*freq[n]*t) - np.imag(surface_fft[n]) * np.sin(2*np.pi*freq[n]*t))

# Plot the noisy signal and reconstructed signal
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(t, HOBO.surface[start_index:end_index])
ax1.set_title('Noisy Signal')
ax2.plot(t, reconstructed_signal)
ax2.set_title('Reconstructed Signal')
plt.show()